# Baseline RAG example

This is a simple example of a baseline RAG application which purpose is to answer questions about the fantasy series [Malazan Universe](https://malazan.fandom.com/wiki/Malazan_Wiki) created by Steven Erikson and Ian C. Esslemont.

First the example will show each step of a baseline RAG pipeline including **Indexing**, **Retrieval** and **Generation**. This is done in order to show the architecture without the abstraction provided by frameworks like LlamaIndex and LangChain.
Then a more "normal" example will be shown using LlamaIndex.

As a vector database, we will use [ChromaDB](https://docs.trychroma.com/), but this can easily be exchanged with other databases.

In this example, we will use the following technologies

- OpenAI API
- ChromaDB
- LlamaIndex


### Setup libraries and environment


In [1]:
#%pip install chromadb llama-index-vector-stores-chroma

In [2]:
import os

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb import Settings
from IPython.display import Markdown, display
from llama_index.core import PromptTemplate, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from openai import OpenAI, AzureOpenAI

import importlib
import util

# Suppose you made some changes to util.py

#importlib.reload(util.helpers)
from util.helpers import create_and_save_md_files, get_malazan_pages, get_theoffice_pages

### Environment variables

For this example you need to use an OpenAI API key. Go to [your API keys](https://platform.openai.com/api-keys) in the OpenAI console to generate one.

Then add the following to a `.env` file in the root of the project.

```
OPENAI_API_KEY=<YOUR_KEY_HERE>
```


In [15]:
from dotenv import load_dotenv

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai_client = AzureOpenAI(
    api_key=OPENAI_API_KEY,  
    api_version="2024-05-01-preview", # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [16]:

#openai_client = AzureOpenAI(api_key=OPENAI_API_KEY)
#openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#    api_key=OPENAI_API_KEY,
#    model_name="text-embedding-3-small"
#)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-ada-002",
    api_type="azure"
)

chroma_client = chromadb.PersistentClient(
    path="./data/baseline-rag/chromadb", settings=Settings(allow_reset=True))

In [5]:
def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return openai_client.embeddings.create(input = [text], model=model).data[0].embedding


## Fetch documents and save them as markdown files

Here we fetch pages from the Fandom Malazan Wiki. These are the documents that we will use as our "knowledge base" in order to supply context to our prompts.

We also pre-process the content in order to be able to add them to our vector database.


In [6]:
pages = get_theoffice_pages()
pages

[<FandomPage 'Michael Scott'>,
 <FandomPage 'Dwight Schrute'>,
 <FandomPage 'Jim Halpert'>,
 <FandomPage 'Pam Beesly'>,
 <FandomPage 'Ryan Howard'>,
 <FandomPage 'Andy Bernard'>,
 <FandomPage 'Angela Martin'>,
 <FandomPage 'Kelly Kapoor'>,
 <FandomPage 'Toby Flenderson'>,
 <FandomPage 'Creed Bratton'>,
 <FandomPage 'Darryl Philbin'>,
 <FandomPage 'Kevin Malone'>,
 <FandomPage 'Meredith Palmer'>,
 <FandomPage 'Oscar Martinez'>,
 <FandomPage 'Phyllis Lapin'>,
 <FandomPage 'Stanley Hudson'>]

In [7]:
create_and_save_md_files(pages)

## Indexing

In this step, we will index the documents in our vector database. This will allow us to retrieve the most relevant documents when we ask a question.

We will use ChromaDB as our vector database and 'text-embedding-3-small' from OpenAI as our embedding model.


#### Fetch and process saved documents

First we need to fetch the documents we saved earlier.

Then we will process the documents in order to add them to our vector database.
The `SimpleDirectoryReader` fetches each section of the markdown file
Then each section is split in to smaller chunks of text and each chunk is embedded using the OpenAI API.


In [8]:
documents = SimpleDirectoryReader('./data/docs').load_data()
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=20)

document_data = []

for document in documents:
    chunks = text_splitter.split_text(document.text)
    for idx, chunk in enumerate(chunks):
        embedding = openai_client.embeddings.create(
            input=chunk, model="text-embedding-ada-002")
        document_data.append({
            "id": f"{document.id_}-{idx}",
            "text": chunk,
            "metadata": document.metadata,
            "embedding": embedding.data[0].embedding
        })

In [9]:
print("embeddings dim", len(document_data[0]['embedding']))
print(document_data[0])

embeddings dim 1536
{'id': '1c50d40f-fbf5-4347-abc8-f33061450e28-0', 'text': 'Andy Bernard\r\n============', 'metadata': {'file_path': 'c:\\Users\\jach\\projekter\\ADVANCED-RAG-EXAMPLES\\data\\docs\\andy-bernard.md', 'file_name': 'andy-bernard.md', 'file_size': 15489, 'creation_date': '2024-05-28', 'last_modified_date': '2024-05-28'}, 'embedding': [-0.015254429541528225, -0.025527821853756905, 0.02062799595296383, -0.02142658643424511, -0.01515968143939972, 0.031916543841362, 0.009840257465839386, -0.02901996113359928, -0.028424402698874474, -0.0029287287034094334, 0.01419866643846035, 0.01216835156083107, 0.025906812399625778, -0.006835392210632563, 0.0005663731717504561, -0.005569829605519772, 0.021616080775856972, 0.028586827218532562, 0.0239983182400465, -6.886678875162033e-06, -0.029588449746370316, 0.024282561615109444, -0.0003003173624165356, -0.018381113186478615, 0.01396856363862753, 0.007437718566507101, 0.027490457519888878, -0.027125000953674316, 0.013481288217008114, -0.03

#### Add documents to ChromaDB


In [10]:
documents = [doc["text"] for doc in document_data]
embeddings = [doc["embedding"] for doc in document_data]
metadatas = [doc["metadata"] for doc in document_data]
ids = [doc["id"] for doc in document_data]

In [17]:
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="theoffice", metadata={"hnsw:space": "cosine"}, embedding_function=openai_ef)

In [18]:
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=ids)

## Retrieval

In this step, we will retrieve the most relevant documents to a given question. We will use the vector database to retrieve the most similar documents to the question.

In order to do this we will use the `text-embedding-3-small` model (**the same model used to index the documents**) from OpenAI to embed the question and then use the vector database to retrieve the most similar documents.

We will retrieve the top 5 documents based on the _cosine similarity_ between the question and the documents. Other similarity metrics can be used as well like squared L2 or inner product.

Change `cosine` to `l2` or `ip` when creating the collection above to try these out.


In [28]:
query = "What are some of the made-up characters impersonated by Michael Scott?"

In [33]:
result = collection.query(query_texts=[query], n_results=5)
context = result["documents"][0]
#display(Markdown(f"------------\n\n{"\n\n------------\n\n".join(context)}"))

formatted_text = "\n\n------------\n\n".join(context)

# Display the formatted markdown
display(Markdown(f"{formatted_text}"))

Alter egos of Michael Scott


Given his proclivity for constantly trying to keep his employees entertained (and coupled with his juvenile personality), Michael has created a variety of different alter egos which he uses for both entertainment, and, at times, educational purposes. Often at times, he uses these characters' names to hide transacting information, and at one point his credit card uses "Michael Scarn", instead of Michael Scott.
Ping ("The Dundies", "The Seminar", "Goodbye, Michael"), a Chinese caricature based on Michael's Chinese food delivery man.
Agent Michael Scarn ("Threat Level Midnight", "The Client", "E-mail Surveillance", referenced in "Product Recall", "Money", "Dinner Party", "Prince Family Paper") is the star of Threat Level Midnight, a derivative spy/action hero screenplay written and illustrated by Michael. He had kept it hidden in his desk drawer until it was discovered by Pam without his knowledge and photocopied so the staff could stage a rollicking reading of it while Michael was on a sales call. A thinly veiled portrayal of himself, Michael also adopts the persona in one session of his improv comedy class, ignoring the rule to base his dialogue on his scene partners, and ultimately shooting everyone in the room, regardless of their participation in the scene. He also uses the alias in another episode to go undercover in a local paper competitor to obtain information from it ("Prince Family Paper").
Prison Mike ("The Convict") wears a purple bandana over his head, speaking in a caricature of New York English, and explains that he is in prison for theft, robbery and kidnapping the president's son for ransom (although he claims, in knee-jerk defensiveness, that he was never caught). He proceeds to paint an awful, and somewhat fanciful, picture of prison life. He states that the scariest part of prison is the Dementors. The character is likely the result of Michael's take on the Scared Straight! documentary series.
Michael the Magic ("Cocktails") attempts to escape from a straightjacket but fails because of a lost key (actually hidden by Jim). Michael's fondness for magic is referenced throughout the series, including an off-screen visit to a children's magic camp. Michael also attempts to utilize a magic prop briefcase (which included a working chainsaw) while speaking at Karen's Utica branch in "Lecture Circuit".

------------

Michael Scott
=============

------------

In the cold open for "Nepotism", he is seen performing numerous, albeit repetitive, magic tricks.
Michael Klump ("Weight Loss", referenced in "Garage Sale") attempts to show that overweight people are people too. Michael wears a partially inflated sumo suit underneath a business suit, and is patterned after the Eddie Murphy character in the Nutty Professor remakes.
Michael Scotch ("New Boss") was co-created when Michael and Dwight were trying to contact David Wallace to complain when Charles Miner cancelled Michael's fifteenth anniversary party because of budget cuts. Michael Scotch is an overly aggressive character who threatens that he has kidnapped David Wallace's son.
Blind Guy McSqueezy ("The Lover") is a character Michael created at his improv class so he could feel up women.
Caleb Crawdad ("Murder") is a Southern persona used for the purposes of a murder mystery game.
Date Mike ("Happy Hour") is a personality Michael takes on when on a date. Michael creates this personality when Jim points out to him that Pam's friend likes him. Up to that point, Michael had been charming and likable, but Date Mike completely ruined things and the friend fled in disgust. Ironically, Michael felt Date Mike was successful because he impressed Donna, while Jim thought Date Mike was a disaster because he didn't know about Donna. Michael says Date Mike is inspired by watching dating competitions, saying " I absorb information, from the strategies of winners. And the losers! Actually I probably learn more from the losers." Date Mike comes off as egotistical and introduces himself with the line "Hi, I'm Date Mike. Nice to meet me. How do you like your eggs in the morning?"
Michael Wonka ("Golden Ticket") is a character of Michael's referencing Willy Wonka from Willy Wonka and the Chocolate Factory. He was used to announce Michael's "Golden Ticket" idea.
Scranton Strangler ("Happy Hour")(Bloopers Season 6) occurs in the same episode as Date Mike, although he only appears in the blooper reel for the sixth season. Michael claims that "Oh, I'm working on a new one. Scranton Strangler... 'Hey, I'm going to kill you...'" The character was cut from the aired version of the episode.
Mike Leno is an interview personality that is a parody of Jay Leno.

------------

Michael Scott


“
A good manager doesn't fire people. He hires people and inspires people. … People, Ryan. And people will never go out of business.
”
—Michael Scott to Ryan Howard, Business School
Michael Gary Scott (born March 15, 1965) is a fictional character on The Office, who is portrayed by Steve Carell and based on David Brent from the British version of the program. Michael is the main character of the series, serving as the Regional Manager of the Scranton branch of a paper distribution company, known as Dunder Mifflin Inc. from seasons 1–7. However, he temporarily leaves Dunder Mifflin to form The Michael Scott Paper Company with Pam Beesly and Ryan Howard toward the end of the 5th season and shares a co-managerial position with Jim Halpert during a 6th season arc from "The Meeting" to "Manager and Salesman". In the end of the 7th season, he proposes to HR representative Holly Flax and moves to Colorado to take care of her aging parents, leaving the manager position to Deangelo Vickers in "Goodbye, Michael", to Andy Bernard in season 8, and ultimately to Dwight Schrute in season 9.

------------

Michael holds the record for the most episodes named after him, with five: "Michael's Birthday", "Michael Scott Paper Company", "Scott's Tots", "Michael's Last Dundies" and "Goodbye, Michael".
If the episodes "Michael Scott Paper Company" and "Scott's Tots", named after his in-universe endeavors and not after him directly, are not counted, he ties with Dwight ("Dwight's Speech", "Dwight K. Schrute, (Acting) Manager" and "Dwight Christmas") and Andy ("Andy's Play", "Angry Andy", "Andy's Ancestry") with three.

## Generation

In this step, we will generate an answer to the question using the retrieved documents as context. We will use the OpenAI API to generate the answer.


In [34]:
prompt = PromptTemplate("""You are a helpful assistant that answers questions about the US tv show known as "The Office" using provided context. 

Question: {query}

Context: 

-----------------------------------
{context}

-----------------------------------

""")
message = prompt.format(query=query, context="\n\n".join(context))
display(Markdown(f"{message}"))

You are a helpful assistant that answers questions about the US tv show known as "The Office" using provided context. 

Question: What are some of the made-up characters impersonated by Michael Scott?

Context: 

-----------------------------------
Alter egos of Michael Scott


Given his proclivity for constantly trying to keep his employees entertained (and coupled with his juvenile personality), Michael has created a variety of different alter egos which he uses for both entertainment, and, at times, educational purposes. Often at times, he uses these characters' names to hide transacting information, and at one point his credit card uses "Michael Scarn", instead of Michael Scott.
Ping ("The Dundies", "The Seminar", "Goodbye, Michael"), a Chinese caricature based on Michael's Chinese food delivery man.
Agent Michael Scarn ("Threat Level Midnight", "The Client", "E-mail Surveillance", referenced in "Product Recall", "Money", "Dinner Party", "Prince Family Paper") is the star of Threat Level Midnight, a derivative spy/action hero screenplay written and illustrated by Michael. He had kept it hidden in his desk drawer until it was discovered by Pam without his knowledge and photocopied so the staff could stage a rollicking reading of it while Michael was on a sales call. A thinly veiled portrayal of himself, Michael also adopts the persona in one session of his improv comedy class, ignoring the rule to base his dialogue on his scene partners, and ultimately shooting everyone in the room, regardless of their participation in the scene. He also uses the alias in another episode to go undercover in a local paper competitor to obtain information from it ("Prince Family Paper").
Prison Mike ("The Convict") wears a purple bandana over his head, speaking in a caricature of New York English, and explains that he is in prison for theft, robbery and kidnapping the president's son for ransom (although he claims, in knee-jerk defensiveness, that he was never caught). He proceeds to paint an awful, and somewhat fanciful, picture of prison life. He states that the scariest part of prison is the Dementors. The character is likely the result of Michael's take on the Scared Straight! documentary series.
Michael the Magic ("Cocktails") attempts to escape from a straightjacket but fails because of a lost key (actually hidden by Jim). Michael's fondness for magic is referenced throughout the series, including an off-screen visit to a children's magic camp. Michael also attempts to utilize a magic prop briefcase (which included a working chainsaw) while speaking at Karen's Utica branch in "Lecture Circuit".

Michael Scott
=============

In the cold open for "Nepotism", he is seen performing numerous, albeit repetitive, magic tricks.
Michael Klump ("Weight Loss", referenced in "Garage Sale") attempts to show that overweight people are people too. Michael wears a partially inflated sumo suit underneath a business suit, and is patterned after the Eddie Murphy character in the Nutty Professor remakes.
Michael Scotch ("New Boss") was co-created when Michael and Dwight were trying to contact David Wallace to complain when Charles Miner cancelled Michael's fifteenth anniversary party because of budget cuts. Michael Scotch is an overly aggressive character who threatens that he has kidnapped David Wallace's son.
Blind Guy McSqueezy ("The Lover") is a character Michael created at his improv class so he could feel up women.
Caleb Crawdad ("Murder") is a Southern persona used for the purposes of a murder mystery game.
Date Mike ("Happy Hour") is a personality Michael takes on when on a date. Michael creates this personality when Jim points out to him that Pam's friend likes him. Up to that point, Michael had been charming and likable, but Date Mike completely ruined things and the friend fled in disgust. Ironically, Michael felt Date Mike was successful because he impressed Donna, while Jim thought Date Mike was a disaster because he didn't know about Donna. Michael says Date Mike is inspired by watching dating competitions, saying " I absorb information, from the strategies of winners. And the losers! Actually I probably learn more from the losers." Date Mike comes off as egotistical and introduces himself with the line "Hi, I'm Date Mike. Nice to meet me. How do you like your eggs in the morning?"
Michael Wonka ("Golden Ticket") is a character of Michael's referencing Willy Wonka from Willy Wonka and the Chocolate Factory. He was used to announce Michael's "Golden Ticket" idea.
Scranton Strangler ("Happy Hour")(Bloopers Season 6) occurs in the same episode as Date Mike, although he only appears in the blooper reel for the sixth season. Michael claims that "Oh, I'm working on a new one. Scranton Strangler... 'Hey, I'm going to kill you...'" The character was cut from the aired version of the episode.
Mike Leno is an interview personality that is a parody of Jay Leno.

Michael Scott


“
A good manager doesn't fire people. He hires people and inspires people. … People, Ryan. And people will never go out of business.
”
—Michael Scott to Ryan Howard, Business School
Michael Gary Scott (born March 15, 1965) is a fictional character on The Office, who is portrayed by Steve Carell and based on David Brent from the British version of the program. Michael is the main character of the series, serving as the Regional Manager of the Scranton branch of a paper distribution company, known as Dunder Mifflin Inc. from seasons 1–7. However, he temporarily leaves Dunder Mifflin to form The Michael Scott Paper Company with Pam Beesly and Ryan Howard toward the end of the 5th season and shares a co-managerial position with Jim Halpert during a 6th season arc from "The Meeting" to "Manager and Salesman". In the end of the 7th season, he proposes to HR representative Holly Flax and moves to Colorado to take care of her aging parents, leaving the manager position to Deangelo Vickers in "Goodbye, Michael", to Andy Bernard in season 8, and ultimately to Dwight Schrute in season 9.

Michael holds the record for the most episodes named after him, with five: "Michael's Birthday", "Michael Scott Paper Company", "Scott's Tots", "Michael's Last Dundies" and "Goodbye, Michael".
If the episodes "Michael Scott Paper Company" and "Scott's Tots", named after his in-universe endeavors and not after him directly, are not counted, he ties with Dwight ("Dwight's Speech", "Dwight K. Schrute, (Acting) Manager" and "Dwight Christmas") and Andy ("Andy's Play", "Angry Andy", "Andy's Ancestry") with three.

-----------------------------------



In [39]:
stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": query}],
    model="gpt-35-turbo",
    stream=True)

output = ""
for chunk in stream:
    if chunk.choices:  # Check if the list is not empty
        output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

1. Prison Mike
2. Date Mike
3. Michael Klump
4. Ping
5. Agent Michael Scarn
6. Willy Wonka
7. Santa
8. The World's Best Boss
9. Todd Packer

In [42]:
stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": message}],
    model="gpt-35-turbo",
    stream=True)

output = ""
for chunk in stream:
    if chunk.choices:  # Check if the list is not empty
        output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

Some of the made-up characters impersonated by Michael Scott include Ping, Agent Michael Scarn, Prison Mike, Michael the Magic, Michael Klump, Michael Scotch, Blind Guy McSqueezy, Caleb Crawdad, Date Mike, Michael Wonka, and Mike Leno. He also briefly mentions a character called the Scranton Strangler in the blooper reel of an episode.

## Normal example using LlamaIndex

In this example, we will use LlamaIndex to abstract the indexing and retrieval steps. This shows how easily the same pipeline can be implemented using LlamaIndex.


In [50]:

#%pip install llama-index-embeddings-azure-openai
#%pip install llama-index-llms-azure-openai

In [55]:
import chromadb
from chromadb import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

# ChromaDB Vector Store
chroma_client = chromadb.PersistentClient(
    path="./data/baseline-rag/chromadb", settings=Settings(allow_reset=True))
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="theoffice", metadata={"hnsw:space": "cosine"})
vector_store = ChromaVectorStore(chroma_collection=collection)


llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name="gpt-35-turbo",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Define the ingestion pipeline to add documents to vector store
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=20),
        embedding,
    ],
    vector_store=vector_store,
)

# Create index with the vector store and using the embedding model
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embedding)

In [56]:
# Fetch documents
documents = SimpleDirectoryReader('./data/docs').load_data()

# Run pipeline
pipeline.run(documents=documents)

print("Indexing complete")

Indexing complete


#### Create base QueryEngine from LlamaIndex


In [57]:
query_engine = index.as_query_engine(llm=llm, verbose=True)

#### Or alternatively, create a CustomQueryEngine


In [75]:
from llama_index.core import PromptTemplate
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer

qa_prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about the US tv show of "The Office" using provided context.
    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {query_str}
    Answer: 
    """,
)


class RAGQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: OpenAI
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)
        context_str = "\n\n".join([n.node.get_content() for n in nodes])
        print("Prompt:\n\n", qa_prompt.format(
            context_str=context_str, query_str=query_str))
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )

        return str(response)


synthesizer = get_response_synthesizer(response_mode="compact")
query_engine = RAGQueryEngine(
    retriever=index.as_retriever(),
    response_synthesizer=synthesizer,
    llm=llm,
    qa_prompt=qa_prompt,
)

In [90]:
queries = [
    "What are some of the made-up characters impersonated by Michael Scott?",
    "Who is the character that is known for his 'That's what she said' jokes in The Office?",
    "Which character loves cats?",
    "A character has a heart attack in an episode of the show. Who is it?",
    "What character lives proudly on a farm?",
    "What happens to Kevin's Famous Chili when he brings it to the office?"
]

query = queries[-1]

response = query_engine.query(query)
display(Markdown(f"{response}"))

Prompt:

 You are a helpful assistant that answers questions about the US tv show of "The Office" using provided context.
    Context information is below.
    ---------------------
    Later, when Jim believes Angela stole his baby and he yells out for someone to stop her from leaving the church, Kevin grabs her purse, believing the baby to be stored in there, but instead finds what he thinks to be "thousands" of scones Angela stole from reception.
In "Viewing Party", when the office is watching a live broadcast of the SWAT surrounding the Scranton Strangler's house, he thinks they "should do that thing where they play the really good music to get him to come out". Kevin also sets Michael down his jealous path when he refers to Gabe as the boss. Later, at the viewing party, he climbs into Gabe's bed with a plate of "pigs in a blanket" hors d'oeuvres and pulls the covers over himself, since he wants to try "pigs in a blanket" while under a blanket.
In the beginning of "WUPHF.com", when

There is no information provided about what happens to Kevin's Famous Chili when he brings it to the office.

## Simplest RAG implementation using LlamaIndex


VectorStoreIndex does not seem to work yet with Azure OpenAI authentification...

In [73]:
llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name="gpt-35-turbo",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

#import openai
##reload(openai)
#openai.api_type = "azure"
#openai.api_base = os.environ['AZURE_OPENAI_ENDPOINT']  # e.g., "https://my-openai-resource.openai.azure.com/"
#openai.api_version = os.environ['OPENAI_API_VERSION']  # e.g., "2024-05-01-preview"
#openai.api_key = os.environ['OPENAI_API_KEY']

# Fetch documents
documents = SimpleDirectoryReader('./data/docs').load_data()

# build VectorStoreIndex that takes care of chunking documents
# and encoding chunks to embeddings for future retrieval
index = VectorStoreIndex.from_documents(documents=documents, 
                                        api_key=os.environ['OPENAI_API_KEY'],
                                        base_url=os.environ['AZURE_API_BASE'],
                                        app_url=os.environ['AZURE_OPENAI_ENDPOINT'])
#index = VectorStoreIndex.from_documents(documents=documents, embed_model=embed_model, llm=llm, verbose=True)

# The QueryEngine class is equipped with the generator
# and facilitates the retrieval and generation steps
query_engine = index.as_query_engine()

# Use your Default RAG
response = query_engine.query(query)
display(Markdown(f"{response}"))

KeyError: 'AZURE_API_BASE'